In [ ]:
import os
import string
import pickle as pkl
import torch
import random
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import LambdaLR
from collections import Counter
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
import os
import numpy as np
data_dir = os.getcwd() + '/aclImdb/'
#combine into a dict and shuffle and then split if possible
#Use random to draw number of contents and delete it after
NUM_TRAIN_POS = 12500
all_train_targets = np.zeros(shape = (2*NUM_TRAIN_POS,), 
                         dtype = int, order = 'F')
all_train_targets[NUM_TRAIN_POS:] = int(1)
all_train_data = []

train_neg_dir = data_dir+'train/neg/'
for file in os.listdir(train_neg_dir):
    file_path = os.path.join(train_neg_dir, file)
    text = open(file_path, 'r')
    text_list = text.read()
    text.close()
    all_train_data.append(text_list)    

train_pos_dir = data_dir+'train/pos/'
for file in os.listdir(train_pos_dir):
    file_path = os.path.join(train_pos_dir, file)
    text = open(file_path, 'r')
    text_list = text.read()
    text.close()
    all_train_data.append(text_list)    

test_targets = np.zeros(shape = (2*NUM_TRAIN_POS,), 
                         dtype = int, order = 'F')
test_targets[NUM_TRAIN_POS:] = int(1)
test_data = []

test_neg_dir = data_dir+'test/neg/'
for file in os.listdir(test_neg_dir):
    #print('The current i is {}, the current file is {}'.format(int(i), str(file)))
    file_path = os.path.join(test_neg_dir, file)
    text = open(file_path, 'r')
    text_list = text.read()
    text.close()
    test_data.append(text_list)    

test_pos_dir = data_dir+'test/pos/'
for file in os.listdir(test_pos_dir):
    file_path = os.path.join(test_pos_dir, file)
    text = open(file_path, 'r')
    text_list = text.read()
    text.close()
    test_data.append(text_list)   
    

In [ ]:
import random
i = random.randint(0,len(test_data)-1)
print('The rand is {}\nAnd the content is\n{}\nThe sentiment is {}\n'.format(i,test_data[i], test_targets[i]))
if i <= 12499:
    print('The content should be file {}\n'.format(os.listdir(test_neg_dir)[i]))
else:
    print('The content should be file {}\n'.format(os.listdir(test_pos_dir)[i-12500]))

In [ ]:
all_train = list(zip(all_train_data, all_train_targets))
random.shuffle(all_train)
all_train_data, all_train_targets = zip(*all_train)
TRAIN_SPLIT = 5000
val_data = list(all_train_data[:TRAIN_SPLIT])
val_targets = np.asarray(all_train_targets[:TRAIN_SPLIT], dtype=int, order = 'F')
train_data = list(all_train_data[TRAIN_SPLIT:])
train_targets = np.asarray(all_train_targets[TRAIN_SPLIT:],dtype = int, order = 'F')

In [ ]:
pkl.dump(train_targets, open("./new_tokens_upper/train_targets.p", "wb"))
pkl.dump(val_targets, open("./new_tokens_upper/val_targets.p", "wb"))
pkl.dump(test_targets, open("./new_tokens_upper/test_targets.p", "wb"))
pkl.dump(train_data, open("./new_tokens_upper/train_data.p", "wb"))
pkl.dump(test_data, open("./new_tokens_upper/test_data.p", "wb"))
pkl.dump(val_data, open("./new_tokens_upper/val_data.p", "wb"))

In [2]:
import spacy
import string
tokenizer = spacy.load('en_core_web_sm')
punctuations = string.punctuation

HTML_STR = '<br />'
QUOTE_MARK = "\'"
punc_list = list(punctuations)
punc_list.remove('\'')#Due to abbreviations, it is actually important to keep the single quote mark

def tokenize_lower_no_punc(input):
    input_copy = input
    input_copy = input_copy.replace(HTML_STR, ' ')
    for punc in punc_list:
        input_copy = input_copy.replace(punc, ' ')
    tokens = tokenizer(input_copy)
    return [token.text.lower() for token in tokens if (not token.text.isspace())]

def tokenize_no_punc(input):
    tokens = tokenizer(input)
    return [token.text for token in tokens if (token.text not in punctuations)]

def tokenize_orig(input):
    tokens = tokenizer(input)
    return [token.text for token in tokens]

#Replace all the punctuations with space, while removing white space when tokenizing, while keeping the upper and lower case
def tokenize_no_punc_better(input):
    input_copy = input
    input_copy = input_copy.replace(HTML_STR, ' ')#Get rid of the HTML string    
    for punc in punc_list:
        input_copy = input_copy.replace(punc, ' ')
    tokens = input_copy.split()
    return [token for token in tokens if (not token.isspace())]

def n_gram_generator(n, tokens):
    n_gram_bag = []
    for j in range(len(tokens)):
        n_gram_bag.append(tuple(tokens[j:j+n]))
        if (j+n) == len(tokens):
            break;
    return n_gram_bag

def tokenize_dataset(n, dataset):
    token_dataset = []
    all_tokens = []
    for sample in dataset:
        tokens = tokenize_no_punc_better(sample)
        n_gram_tokens = n_gram_generator(n, tokens)
        token_dataset.append(n_gram_tokens)
        all_tokens += n_gram_tokens
        
    return token_dataset, all_tokens

#Small test to make sure this works well
#test_str = 'I ate an Apple Today.this is good; I\'m just testing...Hopefully this will work plzzzz'
#test_tokens = tokenize_no_punc_better(test_str)
#n_gram_generator(2, test_tokens)

In [ ]:
#Tokenize the train sets with upper cases
print("Tokenizing train data")
train_data_tokens, all_train_tokens = tokenize_dataset(4, train_data)
print("Tokenizing test data")
test_data_tokens, _ = tokenize_dataset(4, test_data)
print("Tokenizing val data")
val_data_tokens,  _ = tokenize_dataset(4, val_data)


In [ ]:
pkl.dump(train_data_tokens, open("./new_tokens_upper/train_data_tokens_4.p", "wb"))
pkl.dump(test_data_tokens, open("./new_tokens_upper/test_data_tokens_4.p", "wb"))
pkl.dump(val_data_tokens, open("./new_tokens_upper/val_data_tokens_4.p", "wb"))
pkl.dump(all_train_tokens, open("./new_tokens_upper/all_train_data_tokens_4.p", "wb"))


In [ ]:
import pickle as pkl
for i in range(1,5):
    print("Currently tokenizing i = {}".format(int(i)))
    print("Tokenizing train data")
    train_data_tokens, all_train_tokens = tokenize_dataset(i, train_data)
    print("Tokenizing test data")
    test_data_tokens, _ = tokenize_dataset(i, test_data)
    print("Tokenizing val data")
    val_data_tokens,  _ = tokenize_dataset(i, val_data)
    
    train_data_file_name = "train_data_tokens_"
    test_data_file_name = "test_data_tokens_"
    val_data_file_name = "val_data_tokens_"
    all_train_data_file_name = "all_train_data_tokens_"
    
    pkl.dump(train_data_tokens, open(train_data_file_name + str(i) + ".p", "wb"))
    pkl.dump(test_data_tokens, open(test_data_file_name + str(i) + ".p", "wb"))
    pkl.dump(val_data_tokens, open(val_data_file_name + str(i) + ".p", "wb"))
    pkl.dump(all_train_tokens, open(all_train_data_file_name + str(i) + ".p", "wb"))
    

# Reload the old data and pick only those with specific number of tokens

In [ ]:
#Load the old data and pick only n-gram tokens
train_data_tokens_lower = pkl.load(open("./tokens_lower/train_data_tokens_4.p", "rb"))
val_data_tokens_lower = pkl.load(open("./tokens_lower/val_data_tokens_4.p", "rb"))
test_data_tokens_lower = pkl.load(open("./tokens_lower/test_data_tokens_4.p", "rb"))
all_train_tokens_lower = pkl.load(open("./tokens_lower/all_train_data_tokens_4.p", "rb"))
new_train_data_tokens_lower = []
for token_list in train_data_tokens_lower:
    new_token_list = []
    for token in token_list:
        if len(token) == 4:
            new_token_list.append(token)
    new_train_data_tokens_lower.append(new_token_list)
    
new_val_data_tokens_lower = []
for token_list in val_data_tokens_lower:
    new_token_list = []
    for token in token_list:
        if len(token) == 4:
            new_token_list.append(token)
    new_val_data_tokens_lower.append(new_token_list)

new_test_data_tokens_lower = []
for token_list in test_data_tokens_lower:
    new_token_list = []
    for token in token_list:
        if len(token) == 4:
            new_token_list.append(token)
    new_test_data_tokens_lower.append(new_token_list)
    
new_all_train_tokens_lower = []
for token in all_train_tokens_lower:
    if len(token) == 4:
        new_all_train_tokens_lower.append(token)

        
pkl.dump(new_train_data_tokens_lower, open('./new_tokens_lower/train_data_tokens_4.p', 'wb'))
pkl.dump(new_test_data_tokens_lower, open('./new_tokens_lower/test_data_tokens_4.p', 'wb'))
pkl.dump(new_val_data_tokens_lower, open('./new_tokens_lower/val_data_tokens_4.p', 'wb'))
pkl.dump(new_all_train_tokens_lower, open('./new_tokens_lower/all_train_data_tokens_4.p', 'wb'))

# Some function definitions:

In [35]:
PAD_IDX = 0
UNK_IDX = 1

#Build a vocab return id2token and token2id, which is fed on a vocab size
def build_vocab(all_tokens, MAX_VOCAB_SIZE):
    token_counter = Counter(all_tokens)
    vocab, count = zip(*token_counter.most_common(MAX_VOCAB_SIZE))
    id2token = list(vocab)
    token2id = dict(zip(vocab, range(2, 2+len(vocab))))
    id2token = ['<pad>', '<unk>']+id2token
    token2id['<pad>'] = PAD_IDX
    token2id['<unk>'] = UNK_IDX
    return id2token, token2id

#Token2 id dataset which returns an id dataset
def token2id_dataset(tokens_data):
    indices_data = []
    for tokens in tokens_data:
        index_list = [token2id[token] if token in token2id else UNK_IDX for token in tokens]
        indices_data.append(index_list)
    return indices_data

#initialize the MovieReviewDataset
MAX_REVIEW_LENGTH = 300
class MovieReviewDataset(Dataset):
    def __init__(self, data_list, target_list):
        self.data_list = data_list
        self.target_list = target_list
        assert(len(data_list) == len(target_list))
        
    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, index): #retrieves the data tha corresponds to the key 
        content = self.data_list[index][:MAX_REVIEW_LENGTH]
        label = self.target_list[index]
        return [content, len(content), label]
          
def MovieReview_collate_func(batch):
    data_list = []
    label_list = []
    length_list = []
    for datum in batch:
        length_list.append(datum[1])
        label_list.append(datum[2])
    for datum in batch:
        padded_sentence = np.pad(np.array(datum[0]),pad_width=((0,MAX_REVIEW_LENGTH-datum[1])),
                                 mode ="constant", constant_values = 0)
        #Pads after the end with number of max length - datum length
        data_list.append(padded_sentence)
    return [torch.from_numpy(np.array(data_list)), torch.LongTensor(length_list), torch.LongTensor(label_list)] 


class BagOfNGrams(nn.Module):
    def __init__(self, vocab_size, emb_dim):
        super(BagOfNGrams, self).__init__()
        self.embed = nn.Embedding(vocab_size, emb_dim, padding_idx = 0)
        self.linear = nn.Linear(emb_dim,2) # As we only have two categories
        
    def forward(self, data, length):
        out = self.embed(data)
        out = torch.sum(out, dim =1)
        out /= length.view(length.size()[0],1).expand_as(out).float()
        
        out = self.linear(out.float())
        return out

#Test sets for futher testing on the validations sets
def test_model(loader, model):
    correct = 0
    total = 0
    model.eval()
    for data, lengths, labels in loader:
        data_batch, length_batch, label_batch = data, lengths, labels
        outputs = F.softmax(model(data_batch, length_batch), dim=1)
        predicted = outputs.max(1, keepdim=True)[1]
        #print(len(predicted.eq(labels.view_as(predicted)).numpy()))
        total += labels.size(0)
        correct += predicted.eq(labels.view_as(predicted)).sum().item()
    return (100 * correct / total)

def extract_example(loader,model):
    correct = []
    wrong = []
    model.eval()
    for data, lengths, labels in loader:
        data_batch, length_batch, label_batch = data, lengths, labels
        outputs = F.softmax(model(data_batch, length_batch), dim=1)
        predicted = outputs.max(1, keepdim=True)[1]
        pred_array = predicted.eq(labels.view_as(predicted)).numpy()
        for i in range(0,len(pred_array)):
            if pred_array[i] == 1:
                correct.append(str(i))
            else:
                wrong.append(str(i))
            if (len(correct) == 3) & (len(wrong) == 3):
                return correct, wrong
    return correct, wrong


# Tuning for MAX_VOCAB_SIZE

In [ ]:
#Loading the training targets
train_targets = pkl.load(open('./new_tokens_lower/train_targets.p', 'rb'))
test_targets = pkl.load(open('./new_tokens_lower/test_targets.p', 'rb'))
val_targets = pkl.load(open('./new_tokens_lower/val_targets.p', 'rb'))

In [ ]:
#Load the data
train_data_tokens = pkl.load(open('./new_tokens_lower/train_data_tokens_1.p', 'rb'))
val_data_tokens = pkl.load(open('./new_tokens_lower/val_data_tokens_1.p', 'rb'))
test_data_tokens = pkl.load(open('./new_tokens_lower/test_data_tokens_1.p', 'rb'))
all_train_tokens = pkl.load(open('./new_tokens_lower/all_train_data_tokens_1.p','rb'))

In [ ]:

all_vocab_result = []
all_vocab_train_loss = []
all_vocab_eval_acc = []

for MAX_VOCAB_SIZE in range(20000,90000,20000):
    
    #Build the vocab
    print("The current MAX_VOCAB_SIZE IS {}".format(MAX_VOCAB_SIZE))
    id2token, token2id = build_vocab(all_train_tokens, MAX_VOCAB_SIZE);
    
    ##little test
    random_id = random.randint(0, len(id2token)-1)
    random_token = id2token[random_id]
    print("The random token is {}".format(str(random_token)))
    print("To test the id it should be {}, and the result is {}".format(random_id, token2id[random_token]))
    ##End of test
    
    #ID the token datasets
    train_data_id = token2id_dataset(train_data_tokens)
    val_data_id = token2id_dataset(val_data_tokens)
    test_data_id = token2id_dataset(test_data_tokens)

    print("Test the length of token2id_dataset on train data. Lenght should be {}, it is now {}".format(len(train_data_tokens), len(train_data_id)))
    print("Test the length of token2id_dataset on val data. Lenght should be {}, it is now {}".format(len(val_data_tokens), len(val_data_id)))
    print("Test the length of token2id_dataset on test data. Lenght should be {}, it is now {}".format(len(test_data_tokens), len(test_data_id)))

    ##Initialize the MovieReviewDataset object and create batches.
    BATCH_SIZE = 64
    train_dataset = MovieReviewDataset(train_data_id, train_targets)
    train_loader = DataLoader(dataset = train_dataset,batch_size = BATCH_SIZE,
                          collate_fn = MovieReview_collate_func,
                            shuffle = True)

    val_dataset = MovieReviewDataset(val_data_id, val_targets)
    val_loader = DataLoader(dataset = val_dataset, batch_size = BATCH_SIZE,
                       collate_fn = MovieReview_collate_func,
                       shuffle = True)

    test_dataset = MovieReviewDataset(test_data_id,test_targets)
    test_loader = DataLoader(dataset = test_dataset, batch_size = BATCH_SIZE,
                        collate_fn = MovieReview_collate_func,
                        shuffle = False)
    ##End of initializations
    
    ##Build the BagOfNGram Models
    #Needs further tuning
    emb_dim = 100
    model = BagOfNGrams(len(id2token), emb_dim)
    ##End of building models
    
    #Parameter settings -> tuning needed later
    learning_rate= 0.01
    num_epochs = 10
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
    #Parameter seting ended

    #For each vocab sizes, run through and record
    train_loss_val = []
    eval_acc_val = []
    for epoch in range(num_epochs):
        for i, (data, length, labels) in enumerate(train_loader):
            model.train()
            data_batch, length_batch, label_batch = data, length, labels
            optimizer.zero_grad()
            outputs = model(data_batch, length_batch)
            loss = criterion(outputs, label_batch)
            train_loss_val.append(loss.item())
            loss.backward()
            optimizer.step()

            if i > 0 and i% 100 == 0:
                val_acc = test_model(val_loader, model)
                eval_acc_val.append(val_acc)
                print('Epoch: [{}/{}], Step: [{}/{}], Validation Acc: {}'.format( 
                       epoch+1, num_epochs, i+1, len(train_loader), val_acc))

    all_vocab_train_loss.append(train_loss_val)
    all_vocab_eval_acc.append(eval_acc_val)
        
    
    val_acc = test_model(val_loader, model)
    
    curr_vocab_result_list = [val_acc]
    print ("After training for {} epochs".format(num_epochs))
    print ("Val Acc {}".format(val_acc))
    all_vocab_result.append(curr_vocab_result_list)

train_loss_file = "./new_tokens_lower/all_vocab_train_loss_1.p"
eval_acc_file = "./new_tokens_lower/all_vocab_eval_acc_1.p"
result_file = "./new_tokens_lower/vocab_result_1.p"
print("Currently saving files {}, {}, {}".format(train_loss_file, eval_acc_file, result_file))

pkl.dump(all_vocab_train_loss, open(train_loss_file,"wb"))
pkl.dump(all_vocab_eval_acc, open(eval_acc_file,"wb"))
pkl.dump(all_vocab_result, open(result_file,"wb"))



In [ ]:
emb_list = [200,400,600,1000]

In [ ]:

all_emb_result = []
all_emb_train_loss = []
all_emb_eval_acc = []

for emb_dim in emb_list:
    #Build the vocab
    MAX_VOCAB_SIZE = 80000
    print("The current MAX_VOCAB_SIZE IS {}".format(MAX_VOCAB_SIZE))
    id2token, token2id = build_vocab(all_train_tokens, MAX_VOCAB_SIZE);
    
    ##little test
    random_id = random.randint(0, len(id2token)-1)
    random_token = id2token[random_id]
    print("The random token is {}".format(str(random_token)))
    print("To test the id it should be {}, and the result is {}".format(random_id, token2id[random_token]))
    ##End of test
    
    #ID the token datasets
    train_data_id = token2id_dataset(train_data_tokens)
    val_data_id = token2id_dataset(val_data_tokens)
    test_data_id = token2id_dataset(test_data_tokens)

    print("Test the length of token2id_dataset on train data. Lenght should be {}, it is now {}".format(len(train_data_tokens), len(train_data_id)))
    print("Test the length of token2id_dataset on val data. Lenght should be {}, it is now {}".format(len(val_data_tokens), len(val_data_id)))
    print("Test the length of token2id_dataset on test data. Lenght should be {}, it is now {}".format(len(test_data_tokens), len(test_data_id)))

    ##Initialize the MovieReviewDataset object and create batches.
    BATCH_SIZE = 64
    train_dataset = MovieReviewDataset(train_data_id, train_targets)
    train_loader = DataLoader(dataset = train_dataset,batch_size = BATCH_SIZE,
                          collate_fn = MovieReview_collate_func,
                            shuffle = True)

    val_dataset = MovieReviewDataset(val_data_id, val_targets)
    val_loader = DataLoader(dataset = val_dataset, batch_size = BATCH_SIZE,
                       collate_fn = MovieReview_collate_func,
                       shuffle = True)

    test_dataset = MovieReviewDataset(test_data_id,test_targets)
    test_loader = DataLoader(dataset = test_dataset, batch_size = BATCH_SIZE,
                        collate_fn = MovieReview_collate_func,
                        shuffle = False)
    ##End of initializations
    
    ##Build the BagOfNGram Models
    #Needs further tuning
    print('The current emb_dim is {}'.format(emb_dim))
    model = BagOfNGrams(len(id2token), emb_dim)
    ##End of building models
    
    #Parameter settings -> tuning needed later
    learning_rate= 0.01
    num_epochs = 10
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
    #Parameter seting ended

    #For each emb sizes, run through and record
    train_loss_val = []
    eval_acc_val = []
    for epoch in range(num_epochs):
        for i, (data, length, labels) in enumerate(train_loader):
            model.train()
            data_batch, length_batch, label_batch = data, length, labels
            optimizer.zero_grad()
            outputs = model(data_batch, length_batch)
            loss = criterion(outputs, label_batch)
            train_loss_val.append(loss.item())
            loss.backward()
            optimizer.step()

            if i > 0 and i% 100 == 0:
                val_acc = test_model(val_loader, model)
                eval_acc_val.append(val_acc)
                print('Epoch: [{}/{}], Step: [{}/{}], Validation Acc: {}'.format( 
                       epoch+1, num_epochs, i+1, len(train_loader), val_acc))

    all_emb_train_loss.append(train_loss_val)
    all_emb_eval_acc.append(eval_acc_val)
        
    
    val_acc = test_model(val_loader, model)
    
    curr_emb_result_list = [val_acc]
    print ("After training for {} epochs".format(num_epochs))
    print ("Val Acc {}".format(val_acc))
    all_emb_result.append(curr_emb_result_list)

train_loss_file = "./new_tokens_lower/all_emb_train_loss_1.p"
eval_acc_file = "./new_tokens_lower/all_emb_eval_acc_1.p"
result_file = "./new_tokens_lower/emb_result_1.p"
print("Currently saving files {}, {}, {}".format(train_loss_file, eval_acc_file, result_file))

pkl.dump(all_emb_train_loss, open(train_loss_file,"wb"))
pkl.dump(all_emb_eval_acc, open(eval_acc_file,"wb"))
pkl.dump(all_emb_result, open(result_file,"wb"))



# Therefore, based on the validation accuracy, I will choose vocab_size = 80000 and emb_dim = 200 for n= 2,3,4

In [ ]:
all_n_result = []
all_n_train_loss = []
all_n_eval_acc = []

for i in range(2,5):    
    train_data_file_name = './new_tokens_lower/train_data_tokens_' + str(i) +'.p'
    val_data_file_name = './new_tokens_lower/val_data_tokens_'+str(i)+'.p'
    test_data_file_name = './new_tokens_lower/test_data_tokens_'+ str(i) + '.p'
    all_data_file_name = './new_tokens_lower/all_train_data_tokens_'+ str(i) +'.p'
    
    print('Currently loading file {}'.format(train_data_file_name))
    train_data_tokens = pkl.load(open(train_data_file_name, 'rb'))
    print('Currently loading file {}'.format(val_data_file_name))

    val_data_tokens = pkl.load(open(val_data_file_name, 'rb'))
    print('Currently loading file {}'.format(test_data_file_name))
    test_data_tokens = pkl.load(open(test_data_file_name, 'rb'))
    print('Currently loading file {}'.format(all_data_file_name))
    all_train_tokens = pkl.load(open(all_data_file_name,'rb'))
    
    MAX_VOCAB_SIZE = 80000
    print("The current MAX_VOCAB_SIZE IS {}".format(MAX_VOCAB_SIZE))
    id2token, token2id = build_vocab(all_train_tokens, MAX_VOCAB_SIZE);
    
    ##little test
    random_id = random.randint(0, len(id2token)-1)
    random_token = id2token[random_id]
    print("The random token is {}".format(str(random_token)))
    print("To test the id it should be {}, and the result is {}".format(random_id, token2id[random_token]))
    ##End of test
    
    #ID the token datasets
    train_data_id = token2id_dataset(train_data_tokens)
    val_data_id = token2id_dataset(val_data_tokens)
    test_data_id = token2id_dataset(test_data_tokens)

    print("Test the length of token2id_dataset on train data. Lenght should be {}, it is now {}".format(len(train_data_tokens), len(train_data_id)))
    print("Test the length of token2id_dataset on val data. Lenght should be {}, it is now {}".format(len(val_data_tokens), len(val_data_id)))
    print("Test the length of token2id_dataset on test data. Lenght should be {}, it is now {}".format(len(test_data_tokens), len(test_data_id)))

    ##Initialize the MovieReviewDataset object and create batches.
    BATCH_SIZE = 64
    train_dataset = MovieReviewDataset(train_data_id, train_targets)
    train_loader = DataLoader(dataset = train_dataset,batch_size = BATCH_SIZE,
                          collate_fn = MovieReview_collate_func,
                            shuffle = True)

    val_dataset = MovieReviewDataset(val_data_id, val_targets)
    val_loader = DataLoader(dataset = val_dataset, batch_size = BATCH_SIZE,
                       collate_fn = MovieReview_collate_func,
                       shuffle = True)

    test_dataset = MovieReviewDataset(test_data_id,test_targets)
    test_loader = DataLoader(dataset = test_dataset, batch_size = BATCH_SIZE,
                        collate_fn = MovieReview_collate_func,
                        shuffle = False)
    ##End of initializations
    
    ##Build the BagOfNGram Models
    #Needs further tuning
    emb_dim = 200
    print('The current emb_dim is {}'.format(emb_dim))
    model = BagOfNGrams(len(id2token), emb_dim)
    ##End of building models
    
    #Parameter settings -> tuning needed later
    learning_rate= 0.01
    num_epochs = 10
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
    #Parameter seting ended

    #For each vocab sizes, run through and record
    train_loss_val = []
    eval_acc_val = []
    for epoch in range(num_epochs):
        for i, (data, length, labels) in enumerate(train_loader):
            model.train()
            data_batch, length_batch, label_batch = data, length, labels
            optimizer.zero_grad()
            outputs = model(data_batch, length_batch)
            loss = criterion(outputs, label_batch)
            train_loss_val.append(loss.item())
            loss.backward()
            optimizer.step()

            if i > 0 and i% 100 == 0:
                val_acc = test_model(val_loader, model)
                eval_acc_val.append(val_acc)
                print('Epoch: [{}/{}], Step: [{}/{}], Validation Acc: {}'.format( 
                       epoch+1, num_epochs, i+1, len(train_loader), val_acc))

    all_n_train_loss.append(train_loss_val)
    all_n_eval_acc.append(eval_acc_val)
        
    
    val_acc = test_model(val_loader, model)
    
    curr_n_result_list = [val_acc]
    print ("After training for {} epochs".format(num_epochs))
    print ("Val Acc {}".format(val_acc))
    all_n_result.append(curr_n_result_list)

train_loss_file = "./new_tokens_lower/all_n_train_loss_1.p"
eval_acc_file = "./new_tokens_lower/all_n_eval_acc_1.p"
result_file = "./new_tokens_lower/n_result_1.p"
print("Currently saving files {}, {}, {}".format(train_loss_file, eval_acc_file, result_file))

pkl.dump(all_n_train_loss, open(train_loss_file,"wb"))
pkl.dump(all_n_eval_acc, open(eval_acc_file,"wb"))
pkl.dump(all_n_result, open(result_file,"wb"))
    
    

# After the previous rounds, I will use n = 2, max_vocab = 80000, and embedded dimension = 200 to do the following tuning

In [ ]:
learning_rate_list = [0.001, 0.003, 0.01, 0.03, 0.1]


In [ ]:
#Load the data
train_data_tokens = pkl.load(open('./new_tokens_lower/train_data_tokens_2.p', 'rb'))
val_data_tokens = pkl.load(open('./new_tokens_lower/val_data_tokens_2.p', 'rb'))
test_data_tokens = pkl.load(open('./new_tokens_lower/test_data_tokens_2.p', 'rb'))
all_train_tokens = pkl.load(open('./new_tokens_lower/all_train_data_tokens_2.p','rb'))

In [ ]:
all_lr_result = []
all_lr_train_loss = []
all_lr_eval_acc = []

for learning_rate in learning_rate_list:
    #print('The current learning rate is {}'.format(learning_rate))
    #Build the vocab
    MAX_VOCAB_SIZE = 80000
    print("The current MAX_VOCAB_SIZE IS {}".format(MAX_VOCAB_SIZE))
    id2token, token2id = build_vocab(all_train_tokens, MAX_VOCAB_SIZE);
    
    ##little test
    random_id = random.randint(0, len(id2token)-1)
    random_token = id2token[random_id]
    print("The random token is {}".format(str(random_token)))
    print("To test the id it should be {}, and the result is {}".format(random_id, token2id[random_token]))
    ##End of test
    
    #ID the token datasets
    train_data_id = token2id_dataset(train_data_tokens)
    val_data_id = token2id_dataset(val_data_tokens)
    test_data_id = token2id_dataset(test_data_tokens)

    print("Test the length of token2id_dataset on train data. Lenght should be {}, it is now {}".format(len(train_data_tokens), len(train_data_id)))
    print("Test the length of token2id_dataset on val data. Lenght should be {}, it is now {}".format(len(val_data_tokens), len(val_data_id)))
    print("Test the length of token2id_dataset on test data. Lenght should be {}, it is now {}".format(len(test_data_tokens), len(test_data_id)))

    ##Initialize the MovieReviewDataset object and create batches.
    BATCH_SIZE = 64
    train_dataset = MovieReviewDataset(train_data_id, train_targets)
    train_loader = DataLoader(dataset = train_dataset,batch_size = BATCH_SIZE,
                          collate_fn = MovieReview_collate_func,
                            shuffle = True)

    val_dataset = MovieReviewDataset(val_data_id, val_targets)
    val_loader = DataLoader(dataset = val_dataset, batch_size = BATCH_SIZE,
                       collate_fn = MovieReview_collate_func,
                       shuffle = True)

    test_dataset = MovieReviewDataset(test_data_id,test_targets)
    test_loader = DataLoader(dataset = test_dataset, batch_size = BATCH_SIZE,
                        collate_fn = MovieReview_collate_func,
                        shuffle = False)
    ##End of initializations
    
    ##Build the BagOfNGram Models
    #Needs further tuning
    emb_dim = 200
    print('The current emb_dim is {}'.format(emb_dim))
    model = BagOfNGrams(len(id2token), emb_dim)
    ##End of building models
    
    #Parameter settings -> tuning needed later
    #learning_rate= 0.01
    num_epochs = 10
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
    #Parameter seting ended
    print('The learning rate here is {}'.format(learning_rate))
    #For each learning rate, run through and record
    train_loss_val = []
    eval_acc_val = []
    for epoch in range(num_epochs):
        for i, (data, length, labels) in enumerate(train_loader):
            model.train()
            data_batch, length_batch, label_batch = data, length, labels
            optimizer.zero_grad()
            outputs = model(data_batch, length_batch)
            loss = criterion(outputs, label_batch)
            train_loss_val.append(loss.item())
            loss.backward()
            optimizer.step()

            if i > 0 and i% 100 == 0:
                val_acc = test_model(val_loader, model)
                eval_acc_val.append(val_acc)
                print('Epoch: [{}/{}], Step: [{}/{}], Validation Acc: {}'.format( 
                       epoch+1, num_epochs, i+1, len(train_loader), val_acc))

    all_lr_train_loss.append(train_loss_val)
    all_lr_eval_acc.append(eval_acc_val)
        
    
    val_acc = test_model(val_loader, model)
    
    curr_lr_result_list = [val_acc]
    print ("After training for {} epochs".format(num_epochs))
    print ("Val Acc {}".format(val_acc))
    all_lr_result.append(curr_lr_result_list)

train_loss_file = "./new_tokens_lower/all_lr_train_loss.p"
eval_acc_file = "./new_tokens_lower/all_lr_eval_acc.p"
result_file = "./new_tokens_lower/lr_result.p"
print("Currently saving files {}, {}, {}".format(train_loss_file, eval_acc_file, result_file))

pkl.dump(all_lr_train_loss, open(train_loss_file,"wb"))
pkl.dump(all_lr_eval_acc, open(eval_acc_file,"wb"))
pkl.dump(all_lr_result, open(result_file,"wb"))



In [ ]:
#Load the data
train_data_tokens = pkl.load(open('./new_tokens_lower/train_data_tokens_2.p', 'rb'))
val_data_tokens = pkl.load(open('./new_tokens_lower/val_data_tokens_2.p', 'rb'))
test_data_tokens = pkl.load(open('./new_tokens_lower/test_data_tokens_2.p', 'rb'))
all_train_tokens = pkl.load(open('./new_tokens_lower/all_train_data_tokens_2.p','rb'))

In [ ]:
#Build the vocab
MAX_VOCAB_SIZE = 80000
print("The current MAX_VOCAB_SIZE IS {}".format(MAX_VOCAB_SIZE))
id2token, token2id = build_vocab(all_train_tokens, MAX_VOCAB_SIZE);

##little test
random_id = random.randint(0, len(id2token)-1)
random_token = id2token[random_id]
print("The random token is {}".format(str(random_token)))
print("To test the id it should be {}, and the result is {}".format(random_id, token2id[random_token]))
##End of test

#ID the token datasets
train_data_id = token2id_dataset(train_data_tokens)
val_data_id = token2id_dataset(val_data_tokens)
test_data_id = token2id_dataset(test_data_tokens)

print("Test the length of token2id_dataset on train data. Lenght should be {}, it is now {}".format(len(train_data_tokens), len(train_data_id)))
print("Test the length of token2id_dataset on val data. Lenght should be {}, it is now {}".format(len(val_data_tokens), len(val_data_id)))
print("Test the length of token2id_dataset on test data. Lenght should be {}, it is now {}".format(len(test_data_tokens), len(test_data_id)))

##Initialize the MovieReviewDataset object and create batches.
BATCH_SIZE = 64
train_dataset = MovieReviewDataset(train_data_id, train_targets)
train_loader = DataLoader(dataset = train_dataset,batch_size = BATCH_SIZE,
                      collate_fn = MovieReview_collate_func,
                        shuffle = True)

val_dataset = MovieReviewDataset(val_data_id, val_targets)
val_loader = DataLoader(dataset = val_dataset, batch_size = BATCH_SIZE,
                   collate_fn = MovieReview_collate_func,
                   shuffle = True)

test_dataset = MovieReviewDataset(test_data_id,test_targets)
test_loader = DataLoader(dataset = test_dataset, batch_size = BATCH_SIZE,
                    collate_fn = MovieReview_collate_func,
                    shuffle = False)
##End of initializations

##Build the BagOfNGram Models
#Needs further tuning
emb_dim = 200
print('The current emb_dim is {}'.format(emb_dim))
model = BagOfNGrams(len(id2token), emb_dim)
##End of building models

#Parameter settings -> tuning needed later
learning_rate = 0.001
num_epochs = 10
criterion = torch.nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

#Parameter seting ended
print('The learning rate here is {}'.format(learning_rate))
#For each learning rate, run through and record
train_loss_val = []
eval_acc_val = []
for epoch in range(num_epochs):
    for i, (data, length, labels) in enumerate(train_loader):
        model.train()
        data_batch, length_batch, label_batch = data, length, labels
        optimizer.zero_grad()
        outputs = model(data_batch, length_batch)
        loss = criterion(outputs, label_batch)
        train_loss_val.append(loss.item())
        loss.backward()
        optimizer.step()

        if i > 0 and i% 100 == 0:
            val_acc = test_model(val_loader, model)
            eval_acc_val.append(val_acc)
            print('Epoch: [{}/{}], Step: [{}/{}], Validation Acc: {}'.format( 
                   epoch+1, num_epochs, i+1, len(train_loader), val_acc))

#all_lr_train_loss.append(train_loss_val)
#all_lr_eval_acc.append(eval_acc_val)


val_acc = test_model(val_loader, model)

curr_SGD_result_list = [val_acc]
print ("After training for {} epochs".format(num_epochs))
print ("Val Acc {}".format(val_acc))
#all_lr_test_result.append(curr_lr_result_list)

train_loss_file = "./new_tokens_lower/SGD_train_loss.p"
eval_acc_file = "./new_tokens_lower/SGD_eval_acc.p"
result_file = "./new_tokens_lower/SGD_result.p"
#print("Currently saving files {}, {}, {}".format(train_loss_file, eval_acc_file, result_file))

pkl.dump(train_loss_val, open(train_loss_file,"wb"))
pkl.dump(eval_acc_val, open(eval_acc_file,"wb"))
pkl.dump(curr_SGD_result_list, open(result_file,"wb"))



In [ ]:

#Linear Annealing of learning rate
#Build the vocab
MAX_VOCAB_SIZE = 80000
print("The current MAX_VOCAB_SIZE IS {}".format(MAX_VOCAB_SIZE))
id2token, token2id = build_vocab(all_train_tokens, MAX_VOCAB_SIZE);

##little test
random_id = random.randint(0, len(id2token)-1)
random_token = id2token[random_id]
print("The random token is {}".format(str(random_token)))
print("To test the id it should be {}, and the result is {}".format(random_id, token2id[random_token]))
##End of test

#ID the token datasets
train_data_id = token2id_dataset(train_data_tokens)
val_data_id = token2id_dataset(val_data_tokens)
test_data_id = token2id_dataset(test_data_tokens)

print("Test the length of token2id_dataset on train data. Lenght should be {}, it is now {}".format(len(train_data_tokens), len(train_data_id)))
print("Test the length of token2id_dataset on val data. Lenght should be {}, it is now {}".format(len(val_data_tokens), len(val_data_id)))
print("Test the length of token2id_dataset on test data. Lenght should be {}, it is now {}".format(len(test_data_tokens), len(test_data_id)))

##Initialize the MovieReviewDataset object and create batches.
BATCH_SIZE = 64
train_dataset = MovieReviewDataset(train_data_id, train_targets)
train_loader = DataLoader(dataset = train_dataset,batch_size = BATCH_SIZE,
                      collate_fn = MovieReview_collate_func,
                        shuffle = True)

val_dataset = MovieReviewDataset(val_data_id, val_targets)
val_loader = DataLoader(dataset = val_dataset, batch_size = BATCH_SIZE,
                   collate_fn = MovieReview_collate_func,
                   shuffle = True)

test_dataset = MovieReviewDataset(test_data_id,test_targets)
test_loader = DataLoader(dataset = test_dataset, batch_size = BATCH_SIZE,
                    collate_fn = MovieReview_collate_func,
                    shuffle = False)
##End of initializations

##Build the BagOfNGram Models
#Needs further tuning
emb_dim = 200
print('The current emb_dim is {}'.format(emb_dim))
model = BagOfNGrams(len(id2token), emb_dim)
##End of building models

#Parameter settings -> tuning needed later
learning_rate = 0.001
num_epochs = 10
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
#optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

#Linear annealing
lambda_1 = lambda epoch: 0.9 ** epoch
scheduler = LambdaLR(optimizer, lr_lambda=lambda_1)

#Parameter seting ended
print('The learning rate here is {}'.format(learning_rate))
#For each learning rate, run through and record
train_loss_val = []
eval_acc_val = []
for epoch in range(num_epochs):
    scheduler.step()
    for i, (data, length, labels) in enumerate(train_loader):
        model.train()
        data_batch, length_batch, label_batch = data, length, labels
        optimizer.zero_grad()
        outputs = model(data_batch, length_batch)
        loss = criterion(outputs, label_batch)
        train_loss_val.append(loss.item())
        loss.backward()
        optimizer.step()

        if i > 0 and i% 100 == 0:
            val_acc = test_model(val_loader, model)
            eval_acc_val.append(val_acc)
            print('Epoch: [{}/{}], Step: [{}/{}], Validation Acc: {}'.format( 
                   epoch+1, num_epochs, i+1, len(train_loader), val_acc))

#all_lr_train_loss.append(train_loss_val)
#all_lr_eval_acc.append(eval_acc_val)


val_acc = test_model(val_loader, model)

curr_la_result_list = [val_acc]
print ("After training for {} epochs".format(num_epochs))
print ("Val Acc {}".format(val_acc))
#all_lr_test_result.append(curr_lr_result_list)

train_loss_file = "./new_tokens_lower/la_train_loss.p"
eval_acc_file = "./new_tokens_lower/la_eval_acc.p"
result_file = "./new_tokens_lower/la_result.p"
#print("Currently saving files {}, {}, {}".format(train_loss_file, eval_acc_file, result_file))

pkl.dump(train_loss_val, open(train_loss_file,"wb"))
pkl.dump(eval_acc_val, open(eval_acc_file,"wb"))
pkl.dump(curr_la_result_list, open(result_file,"wb"))



In [32]:
#Load the data
train_data_tokens = pkl.load(open('./new_tokens_lower/train_data_tokens_2.p', 'rb'))
val_data_tokens = pkl.load(open('./new_tokens_lower/val_data_tokens_2.p', 'rb'))
test_data_tokens = pkl.load(open('./new_tokens_lower/test_data_tokens_2.p', 'rb'))
all_train_tokens = pkl.load(open('./new_tokens_lower/all_train_data_tokens_2.p','rb'))

In [33]:
train_targets = pkl.load(open('./new_tokens_lower/train_targets.p', 'rb'))
test_targets = pkl.load(open('./new_tokens_lower/test_targets.p', 'rb'))
val_targets = pkl.load(open('./new_tokens_lower/val_targets.p', 'rb'))

In [36]:
#Final Model Train to get the test results
#Build the vocab
MAX_VOCAB_SIZE = 80000
print("The current MAX_VOCAB_SIZE IS {}".format(MAX_VOCAB_SIZE))
id2token, token2id = build_vocab(all_train_tokens, MAX_VOCAB_SIZE);

##little test
random_id = random.randint(0, len(id2token)-1)
random_token = id2token[random_id]
print("The random token is {}".format(str(random_token)))
print("To test the id it should be {}, and the result is {}".format(random_id, token2id[random_token]))
##End of test

#ID the token datasets
train_data_id = token2id_dataset(train_data_tokens)
val_data_id = token2id_dataset(val_data_tokens)
test_data_id = token2id_dataset(test_data_tokens)

print("Test the length of token2id_dataset on train data. Lenght should be {}, it is now {}".format(len(train_data_tokens), len(train_data_id)))
print("Test the length of token2id_dataset on val data. Lenght should be {}, it is now {}".format(len(val_data_tokens), len(val_data_id)))
print("Test the length of token2id_dataset on test data. Lenght should be {}, it is now {}".format(len(test_data_tokens), len(test_data_id)))

##Initialize the MovieReviewDataset object and create batches.
BATCH_SIZE = 64
train_dataset = MovieReviewDataset(train_data_id, train_targets)
train_loader = DataLoader(dataset = train_dataset,batch_size = BATCH_SIZE,
                      collate_fn = MovieReview_collate_func,
                        shuffle = True)

val_dataset = MovieReviewDataset(val_data_id, val_targets)
val_loader = DataLoader(dataset = val_dataset, batch_size = BATCH_SIZE,
                   collate_fn = MovieReview_collate_func,
                   shuffle = True)

test_dataset = MovieReviewDataset(test_data_id,test_targets)
test_loader = DataLoader(dataset = test_dataset, batch_size = BATCH_SIZE,
                    collate_fn = MovieReview_collate_func,
                    shuffle = False)
##End of initializations

##Build the BagOfNGram Models
#Needs further tuning
emb_dim = 200
print('The current emb_dim is {}'.format(emb_dim))
model = BagOfNGrams(len(id2token), emb_dim)
##End of building models

#Parameter settings 
learning_rate = 0.001
num_epochs = 10
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
#optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

#Linear annealing
lambda_1 = lambda epoch: 0.9 ** epoch
scheduler = LambdaLR(optimizer, lr_lambda=lambda_1)

#Parameter seting ended
print('The learning rate here is {}'.format(learning_rate))
#For each learning rate, run through and record
train_loss_val = []
eval_acc_val = []
for epoch in range(num_epochs):
    scheduler.step()
    for i, (data, length, labels) in enumerate(train_loader):
        model.train()
        data_batch, length_batch, label_batch = data, length, labels
        optimizer.zero_grad()
        outputs = model(data_batch, length_batch)
        loss = criterion(outputs, label_batch)
        train_loss_val.append(loss.item())
        loss.backward()
        optimizer.step()

        if i > 0 and i% 100 == 0:
            val_acc = test_model(val_loader, model)
            eval_acc_val.append(val_acc)
            print('Epoch: [{}/{}], Step: [{}/{}], Validation Acc: {}'.format( 
                   epoch+1, num_epochs, i+1, len(train_loader), val_acc))

#all_lr_train_loss.append(train_loss_val)
#all_lr_eval_acc.append(eval_acc_val)


val_acc = test_model(val_loader, model)
test_acc = test_model(test_loader, model)

curr_result_list = [val_acc, test_acc]
#print ("After training for {} epochs".format(num_epochs))
#print ("Val Acc {}".format(val_acc))
#print ("Test Acc {}".format(test_acc))
#all_lr_test_result.append(curr_lr_result_list)

#train_loss_file = "./new_tokens_upper/train_loss.p"
#eval_acc_file = "./new_tokens_upper/eval_acc.p"
#test_file = "./new_tokens_upper/test_result.p"
#print("Currently saving files {}, {}, {}".format(train_loss_file, eval_acc_file, test_file))

#pkl.dump(train_loss_val, open(train_loss_file,"wb"))
#pkl.dump(eval_acc_val, open(eval_acc_file,"wb"))
#pkl.dump(curr_la_result_list, open(test_file,"wb"))



The current MAX_VOCAB_SIZE IS 80000
The random token is ('2', 'that')
To test the id it should be 44758, and the result is 44758
Test the length of token2id_dataset on train data. Lenght should be 20000, it is now 20000
Test the length of token2id_dataset on val data. Lenght should be 5000, it is now 5000
Test the length of token2id_dataset on test data. Lenght should be 25000, it is now 25000
The current emb_dim is 200
The learning rate here is 0.001
Epoch: [1/10], Step: [101/313], Validation Acc: 56.92
Epoch: [1/10], Step: [201/313], Validation Acc: 66.88
Epoch: [1/10], Step: [301/313], Validation Acc: 75.78
Epoch: [2/10], Step: [101/313], Validation Acc: 82.16
Epoch: [2/10], Step: [201/313], Validation Acc: 83.84
Epoch: [2/10], Step: [301/313], Validation Acc: 85.18
Epoch: [3/10], Step: [101/313], Validation Acc: 85.78
Epoch: [3/10], Step: [201/313], Validation Acc: 86.72
Epoch: [3/10], Step: [301/313], Validation Acc: 86.92
Epoch: [4/10], Step: [101/313], Validation Acc: 87.32
Epoc

In [60]:
val_dataset = MovieReviewDataset(val_data_id, val_targets)
val_loader = DataLoader(dataset = val_dataset, batch_size = BATCH_SIZE,
                   collate_fn = MovieReview_collate_func,
                   shuffle = False)

In [4]:
import pickle as pkl
import os

In [6]:
os.getcwd()

'/Users/Hengyu/Desktop/MasterWork/1011'

In [23]:
result_file = pkl.load(open('./HW1/new_tokens_lower/test_lr_result.p','rb'))

In [28]:
result_file[4]

[84.6, 83.452]

# Visualizations and graph plotting

In [ ]:
train_plot_file = pkl.load(open('./new_tokens_lower/all_vocab_train_loss_1.p', 'rb'))
val_plot_file = pkl.load(open('./new_tokens_lower/all_vocab_eval_acc_1.p', 'rb'))

In [ ]:
vocab_list = [20000,40000,60000,80000]
fig, ax = plt.subplots(nrows = 2, ncols = 4, figsize = (24,8))
for i in range(0,4):
    ax[0,i].plot(train_plot_file[i])
    ax[1,i].plot(val_plot_file[i])
    ax[0,i].set_xlabel("Training data size")
    ax[1,i].set_xlabel("Epochs")
    ax[0,i].set_ylabel("Training loss")
    ax[1,i].set_ylabel("Evaluation accuracy")
    ax[0,i].set_title("Vocab Size: {}".format(vocab_list[i]))

fig.savefig('./new_tokens_lower/vocab_plot.png')

In [ ]:
train_plot_file = pkl.load(open('./new_tokens_lower/all_lr_train_loss.p', 'rb'))
val_plot_file = pkl.load(open('./new_tokens_lower/all_lr_eval_acc.p', 'rb'))
extra_train_file = pkl.load(open('./new_tokens_upper/train_loss.p', 'rb'))
extra_val_file = pkl.load(open('./new_tokens_upper/eval_acc.p', 'rb'))

In [ ]:
#optimizer_list = ['Constant', 'Adam']
fig, ax = plt.subplots(nrows = 2, ncols = 2, figsize = (10,5))
#for i in range(1,2):
#ax[0,0].plot(extra_train_file[0], c='#1f77b4')
#ax[1,0].plot(extra_val_file[0], c='#1f77b4')
#ax[0,0].set_title('Bag of N gram N = 1') 
#ax[0,i].plot(train_plot_file[i-1])
#ax[1,i].plot(val_plot_file[i-1])
#ax[0,i].set_xlabel("Training data size")
#ax[1,i].set_xlabel("Epochs")
#ax[0,i].set_ylabel("Training loss")
#ax[1,i].set_ylabel("Evaluation accuracy")
#ax[1,i].set_ylim([60,90])
#ax[0,i].set_title("Bag of N grams, N = {}".format(vocab_list[i-1]))
ax[0,0].plot(train_plot_file[0])
ax[1,0].plot(val_plot_file[0])
ax[0,1].plot(extra_train_file)
ax[1,1].plot(extra_val_file)
ax[0,0].set_title('Tokenization removing punctuation and lower case')
ax[0,1].set_title('Tokenization upper case')
ax[0,0].set_xlabel('Training data size')
ax[0,1].set_xlabel('Training data size')
ax[1,0].set_xlabel('Epochs')
ax[1,1].set_xlabel('Epochs')
ax[0,0].set_ylabel('Training loss')
ax[0,1].set_ylabel('Training loss')
ax[1,0].set_ylabel('Evaluation accuracy')
#ax[1,0].set_ylim = [50,90]
#ax[1,1].set_ylim = [50,150]
ax[1,1].set_ylabel('Evaluation accuracy')

fig.savefig('./new_tokens_lower/tk_plot.png')

In [ ]:
file1 = pkl.load(open('all_eval_acc_2_vocab_100', 'rb'))
file2 = pkl.load(open('all_eval_acc_2_vocab_200.p', 'rb'))
file3 = pkl.load(open('all_eval_acc_2_vocab_300.p', 'rb'))
file4 = pkl.load(open('all_eval_acc_2_vocab_400.p', 'rb'))

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 8, figsize=(48,4))
for i in range(0,8):
    ax[i].plot(file1[i], label = 'emb_dim = 100')
    ax[i].plot(file2[i], label = 'emb_dim = 200')
    ax[i].plot(file3[i], label = 'emb_dim = 300')
    ax[i].plot(file4[i], label = 'emb_dim = 400')
    ax[i].set_title('The vocab size = {}'.format(10000*(i+1)))
    ax[i].set_xlabel('Epochs')
    ax[i].set_ylabel('Evaluation Accuracy')
    ax[i].legend()
fig.savefig('./new_tokens_lower/ultimate_graph.png')